## Run the LISAandNEA process end-to-end for a bunch of datasets and classifiers

To be parameterised with Papermill, a notebook needs a cell tagged "parameters"

1. Open the notebook in VSC's built-in text editor 
2. Find the cell 
3. Replace ```"metadata": {},``` with ```"metadata": { "tags": [ "parameters" ] },```
4. Save the notebook

In [ ]:
import logging
import papermill as pm
from datetime import datetime

In [ ]:
# Suppress 'Black is not installed' warning from papermill
# Credit: https://github.com/ploomber/ploomber/pull/831/files

class IgnoreBlackWarning(logging.Filter):
    def filter(self, record):
        return 'Black is not installed' not in record.msg

logging.getLogger("papermill.translators").addFilter(IgnoreBlackWarning())

In [ ]:
all_classifiers = ['cnn', 'fcn', 'resnet11']
epochs = 1000
all_df = [1, 2, 3, 4, 5, 6, 7]

In [ ]:

for df_iter in all_df:
    
    # Processing loop for dataset N
    dataset_name = f'df{df_iter}'
    timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    run_id = f'{dataset_name}_{timestamp}'

    # Run lisadatasets3.ipynb to define the dataset's instances
    print(f'Running lisadatasets for {dataset_name} at {datetime.today()}')
    pm.execute_notebook(
        'lisadatasets3.ipynb',
        f'lisadatasets3_{dataset_name}.ipynb',
        parameters = dict(
            df_iter = df_iter,
            save_file = True
        )
    )

    # Run lisaglitches6.ipynb to create simulated telemetry
    # in 3 chunks to avoid kernel panic at ~2000+ instances
    for chunk in range(3):

        print(f'Running lisaglitches5 for {dataset_name} chunk {chunk} at {datetime.today()}')
        pm.execute_notebook(
            'lisaglitches6.ipynb',
            f'lisaglitches6_{dataset_name}_{chunk}.ipynb',
            parameters = dict(
                dataset_name = dataset_name,
                timestamp = timestamp,
                first_instance = 1000*chunk,
                instances_to_do = 1000
            )
        )
    
    # Run lisa_to_tsc2.ipynb to create dataset
    print(f'Running lisa_to_tsc for {dataset_name} at {datetime.today()}')
    pm.execute_notebook(
        'lisa_to_tsc2.ipynb',
        f'lisa_to_tsc2_{dataset_name}.ipynb',
        parameters = dict(
            run_id = run_id
        )
    )

    dataset_name = f'{dataset_name}_{timestamp}'

    # Run tsc_generic2.ipynb to train and test the classifiers
    for classifier_name in all_classifiers:
        
        print(f'Running tsc_generic for {dataset_name} with {classifier_name} at {datetime.today()}')
        pm.execute_notebook(
            'tsc_generic2.ipynb',
            f'tsc_generic2_{dataset_name}_{classifier_name}.ipynb',
            parameters = dict(
                dataset_name = dataset_name,
                classifier_name = classifier_name,
                epochs = epochs
            )
        )        